In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import scipy

In [ ]:
from scipy.fft import fft, ifft, fftfreq

In [ ]:
# from definitions import (
#     NUM_BLOCKS,
#     NUM_TARGETS,
#     NUM_SAMPLES,
#     NUM_ELECTRODES,
#     SAMPLE_FREQ,
#     SAMPLE_T,
#     TARGET_FREQUENCY,
#     TARGET_PHASE,
#     ELECTRODE_INDEX
# )

In [ ]:
from ssvepcca.utils import load_mat_to_pandas, get_harmonic_columns

In [ ]:
mat_data = scipy.io.loadmat("dataset_chines/S2.mat")

In [ ]:
# mat_data["data"]

In [ ]:
df = load_mat_to_pandas("dataset_chines/S2.mat").set_index(["block", "target", "time_ms"])
dataset = mat_data["data"].T

In [ ]:
(df.loc[0,0].loc[0:1000]["electrode_54"].plot())

In [ ]:
# df[["electrode_54",
#     "electrode_55",
#     "electrode_56",
#     "electrode_57",
#     "electrode_58",
#     "electrode_59"]].mean(axis=1).loc[0,1].plot()

## Test model on the dataset

In [ ]:
import ssvepcca.learners as learners
import ssvepcca.pipelines as pipelines
import ssvepcca.parameters as parameters
import ssvepcca.definitions as definitions
import ssvepcca.utils as utils

In [ ]:
dataset.shape

In [ ]:
model_single = learners.CCASingleComponent(
    electrodes_name=parameters.electrode_list_fbcca,
    start_time_index=0,
    stop_time_index=1500,
)

In [ ]:
elecs = tuple(utils.electrodes_name_to_index(parameters.electrode_list_fbcca))
df = dataset[0, 0, :, :]
h = model_single.harmonic_column(8)

df[:, elecs].shape, h.shape

In [ ]:
cca_model = learners.CCACorrelation(n_components=1, max_iter=1000, scale=False)

In [ ]:
cca_model.fit_correlation(df[:, elecs], h)

In [ ]:
model_single = learners.CCASingleComponent(
    electrodes_name=["O1"],
    start_time_index=0,
    stop_time_index=1500,
)

result_single = pipelines.test_fit_predict(dataset, model_single)

In [ ]:
utils.electrodes_name_to_index(parameters.electrode_list_fbcca)

In [ ]:
l = parameters.electrode_list_fbcca

In [ ]:
l

In [ ]:
model = learners.CCASingleComponent(
    electrodes_name=parameters.electrode_list_fbcca,
    start_time_index=0,
    stop_time_index=1500,
)

result_all = pipelines.test_fit_predict(dataset, model)

In [ ]:
utils.eval_accuracy(result_all[0])

## Develop pipeline to k-fold cv a learner that needs training 

In [ ]:
dataset[:-1, :, :, :]

In [ ]:
target = np.zeros((5, 40))

In [ ]:
for i in range(target.shape[1]): target[:, i] = i 

In [ ]:
target.reshape(-1).shape

In [ ]:
dataset[:-1, :, :, :].shape

In [ ]:
dataset.ndim

In [ ]:
get_harmonic_columns(8).shape

In [ ]:
harmonic_tensor = get_harmonic_columns(8)[np.newaxis, :, :].repeat(3, axis=0)

In [ ]:
harmonic_tensor.shape

In [ ]:
harmonic_tensor.reshape(-1, 6)

In [ ]:
ccafixed = learners.CCAFixedCoefficients(
    electrodes_name=parameters.electrode_list_fbcca
)

In [ ]:
ccaoriginal = learners.CCASingleComponent(
    electrodes_name=parameters.electrode_list_fbcca
)

In [ ]:
dataset.shape

In [ ]:
dataset[..., 0, 0, 0, 0].shape

In [ ]:
ccafixed.fit(dataset[:-1, ...])

In [ ]:
ccafixed.predict(
    dataset[-1, 39, ...]
)[0]

In [ ]:
for i in range(40):
    p = ccaoriginal.predict_proba(dataset[-1, i, ...]).argmax()
    print(i, p, i==p)

In [ ]:
for i in range(40):
    p = ccafixed.predict_proba(dataset[-1, i, ...]).argmax()
    print(i, p, i==p)

In [ ]:
r = pipelines.k_fold_predict(dataset, learners.CCAFixedCoefficients(electrodes_name=parameters.electrode_list_fbcca))
utils.eval_accuracy(r[0])

In [ ]:
r = pipelines.k_fold_predict(dataset, learners.CCAFixedCoefficients(
    electrodes_name=parameters.electrode_list_fbcca,
    start_time_index=125,
    stop_time_index=125 + 250 * 3,
))
utils.eval_accuracy(r[0])

In [ ]:
r = pipelines.test_fit_predict(dataset, learners.CCASingleComponent(electrodes_name=parameters.electrode_list_fbcca))
utils.eval_accuracy(r[0])

In [ ]:
r = pipelines.test_fit_predict(dataset, learners.CCASingleComponent(
    electrodes_name=parameters.electrode_list_fbcca,
    start_time_index=125,
    stop_time_index=1375,
))
utils.eval_accuracy(r[0])

In [ ]:
r = pipelines.test_fit_predict(dataset, learners.CCASingleComponent(
    electrodes_name=parameters.electrode_list_fbcca,
    start_time_index=125,
    stop_time_index=125 + 250 * 3,
))
utils.eval_accuracy(r[0])

## Develop pipeline to test model on all subjects

In [ ]:
sbj = 1
data = utils.load_mat_data_array(f"dataset_chines/S{sbj}.mat")

In [ ]:
definitions.NUM_SUBJECTS

In [ ]:
results = dict(
    predictions = [],
    accuracy = [],
)

for subject_num in range(1, definitions.NUM_SUBJECTS + 1):
    
    dataset_path = f"dataset_chines/S{subject_num}.mat"
    dataset = utils.load_mat_data_array(dataset_path)
    
    model = learners.CCASingleComponent(
        electrodes_name=parameters.electrode_list_fbcca,
        start_time_index=0,
        stop_time_index=1500,
    )
    
    pred = pipelines.test_fit_predict(dataset, model)
    accuracy = utils.metric_accuracy(pred)
    
    results["predictions"].append(pred)
    results["accuracy"].append(accuracy)


In [ ]:
results["predictions"][0]

## Filterbank

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt

In [ ]:
def mfreqz(b, a, Fs):
   
    # Compute frequency response of the filter
    # using signal.freqz function
    wz, hz = signal.freqz(b, a)
 
    # Calculate Magnitude from hz in dB
    Mag = 20*np.log10(abs(hz))
 
    # Calculate phase angle in degree from hz
    Phase = np.unwrap(np.arctan2(np.imag(hz), np.real(hz)))*(180/np.pi)
     
    # Calculate frequency in Hz from wz
    Freq = wz*Fs/(2*np.pi)
     
    # Plot filter magnitude and phase responses using subplot.
    fig = plt.figure(figsize=(10, 6))
 
    # Plot Magnitude response
    sub1 = plt.subplot(2, 1, 1)
    sub1.plot(Freq, Mag, 'r', linewidth=2)
    sub1.axis([1, Fs/2, -100, 5])
    sub1.set_title('Magnitude Response', fontsize=20)
    sub1.set_xlabel('Frequency [Hz]', fontsize=20)
    sub1.set_ylabel('Magnitude [dB]', fontsize=20)
    sub1.grid()
 
    # Plot phase angle
    sub2 = plt.subplot(2, 1, 2)
    sub2.plot(Freq, Phase, 'g', linewidth=2)
    sub2.set_ylabel('Phase (degree)', fontsize=20)
    sub2.set_xlabel(r'Frequency (Hz)', fontsize=20)
    sub2.set_title(r'Phase response', fontsize=20)
    sub2.grid()
 
    plt.subplots_adjust(hspace=0.5)
    fig.tight_layout()
    plt.show()

In [ ]:
fs = 250
fcuts = [8, 16]

b, a = signal.cheby1(
    N=6,
    rp=0.5,
    Wn=fcuts,
    btype="bandpass",
    analog=False,
    output="ba",
    fs=fs
)

In [ ]:
mfreqz(b, a, fs)

In [ ]:
x_original = dataset[0, 0, :, 53]

In [ ]:
signal.filtfilt?

In [ ]:
x_filtered = signal.filtfilt(b=b, a=a, x=x_original)
x_filtered_gust = signal.filtfilt(b=b, a=a, x=x_original, padtype="odd")

In [ ]:
N = 1500 # Number of sample points
T = 1/250 # sample spacing 

fr = fftfreq(N, T)[:N//4]

xf_original = fft(x_original)
xf_filtered = fft(x_filtered)
xf_filtered_gust = fft(x_filtered_gust)


plt.semilogy(fr, 2.0/N * np.abs(xf_original[:N//4]))
plt.semilogy(fr, 2.0/N * np.abs(xf_filtered[:N//4]))
plt.semilogy(fr, 2.0/N * np.abs(xf_filtered_gust[:N//4]))
plt.grid()
plt.show()

In [ ]:
fbcca_model = learners.FBCCA(
    electrodes_name=parameters.electrode_list_fbcca,
    num_harmonics=5,
    fb_num_subband=10,
    fb_fundamental_freq=8,
    fb_upper_bound_freq=88,
)

In [ ]:
fb_proba = fbcca_model.predict_proba(dataset[0,0,...])

In [ ]:
fb_proba.shape

In [ ]:
r = pipelines.test_fit_predict(dataset, fbcca_model)
utils.eval_accuracy(r[0])

## develop FBCCA with SS

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from ssvepcca.utils import load_mat_data_array
from ssvepcca.pipelines import test_fit_predict, k_fold_predict
from ssvepcca.learners import (
    CCASingleComponent, FilterbankCCA, CCAFixedCoefficients, CCAMultiComponent, AlternativeFBCCA,
    CCASpatioTemporal, FBCCAFixedCoefficients, CCASpatioTemporalFixed,
    FBSpatioTemporalCCA
)
from ssvepcca.parameters import electrode_list_fbcca

In [ ]:
input_data = load_mat_data_array("dataset_chines/S2.mat")

In [ ]:
model = FBSpatioTemporalCCA(
    electrodes_name=electrode_list_fbcca,
    start_time_index=125,
    stop_time_index=875,
    num_harmonics=5,
    fb_num_subband=10,
    fb_fundamental_freq=8,
    fb_upper_bound_freq=88,
    window_gap=0,
    window_length=1,
)    

In [ ]:
model.harmonic_column(8).shape

In [ ]:
input_data.shape

In [ ]:
input_measurement = input_data[0, 0, :, :]

In [ ]:
input_measurement.shape

In [ ]:
len(electrode_list_fbcca)

In [ ]:
res = model.feature_extractor(input_measurement)

In [ ]:
res.shape

In [ ]:
res[:, 9].shape

In [ ]:
plt.plot(res[0, 1:, 1] )

In [ ]:
plt.plot(res[0, :-1, 0])

In [ ]:
plt.plot(res[0, 1:, 1] - res[0, :-1, 0])

In [ ]:
plt.plot(res[0,:,7])

In [ ]:
model.predict_proba(input_measurement)

In [ ]:
model.predict(input_measurement)

In [ ]:
plt.plot(input_measurement[model.start_time_index: model.stop_time_index].mean(axis=1))

In [ ]:
plt.plot(input_measurement[model.start_time_index: model.stop_time_index, model.electrodes_index].mean(axis=1))

In [ ]:
plt.plot(input_measurement[model.start_time_index: model.stop_time_index, model.electrodes_index][:, 7])

In [ ]:
plt.plot(
    input_measurement[model.start_time_index: model.stop_time_index, model.electrodes_index][:, 0]
)

In [ ]:
model.feature_extractor(input_measurement).shape

In [ ]:
model.feature_extractor(input_measurement).shape

In [ ]:
from ssvepcca.learners import FBSpatioTemporalCCAFixed
modelfixed = FBSpatioTemporalCCAFixed(
    electrodes_name=electrode_list_fbcca,
    start_time_index=125,
    stop_time_index=875,
    num_harmonics=5,
    fb_num_subband=10,
    fb_fundamental_freq=8,
    fb_upper_bound_freq=88,
    window_gap=3,
    window_length=1,
)    

In [ ]:
input_data[:4, ...].shape

In [ ]:
modelfixed.fit(input_data[:4, ...])

In [ ]:
input_data[4, ...].shape

In [ ]:
modelfixed.predict_proba(input_data[4, 0, ...])

In [ ]:
input_data[..., 0,:,:,:].shape

In [ ]:
preproc = modelfixed.fit(input_data[:4, ...])

In [ ]:
preproc

In [ ]:
preproc2 = preproc.transpose([1, 0, 2, 3])

In [ ]:
preproc2.reshape(preproc2.shape[0], -1, preproc2.shape[-1]).shape

In [ ]:
preproc2.shape

In [ ]:
(preproc2.shape[0],) + preproc2.shape[-2:]

## Develop windowed training data

In [ ]:
START_TIME_INDEX = 125
STOP_TIME_INDEX = 875
NUM_SAMPLES = 1500
USEFUL_NUM_SAMPLES = NUM_SAMPLES - 125

extra_offset = 125
step_size = 125

In [ ]:
input_data.shape

In [ ]:
window_size = (STOP_TIME_INDEX - START_TIME_INDEX) + extra_offset
window_size

In [ ]:
window_size_seconds = window_size / 250
window_size_seconds

In [ ]:
USEFUL_NUM_SAMPLES

In [ ]:
window_size

In [ ]:
window_size + step_size * 4

In [ ]:
window_size / 250

In [ ]:
step_size * 4 / 250

In [ ]:
window_size = 875

In [ ]:
steps = (USEFUL_NUM_SAMPLES - window_size) // step_size + 1

In [ ]:
steps

In [ ]:
for step in range(steps):
    window_start_time = step * step_size
    window_stop_time = window_size + step * step_size
    print(window_start_time/250, window_stop_time/250)

In [ ]:
a = input_data[:, :, window_start_time:window_stop_time, :]
b = input_data[:, :, window_start_time+125:window_stop_time+125, :]

In [ ]:
np.concatenate([a, b], axis=2).shape

In [ ]:
window_size, window_size/250